In [1]:
import deep_predictor
from collections.abc import Mapping
import torch
import scanpy as sc


Global seed set to 0
/home/gaurav/anaconda3/envs/project/lib/python3.10/site-packages/pytorch_lightning/utilities/warnings.py:53: LightningDeprecationWarning: pytorch_lightning.utilities.warnings.rank_zero_deprecation has been deprecated in v1.6 and will be removed in v1.8. Use the equivalent function from the pytorch_lightning.utilities.rank_zero module instead.
  new_rank_zero_deprecation(
/home/gaurav/anaconda3/envs/project/lib/python3.10/site-packages/pytorch_lightning/utilities/warnings.py:58: LightningDeprecationWarning: The `pytorch_lightning.loggers.base.rank_zero_experiment` is deprecated in v1.7 and will be removed in v1.9. Please use `pytorch_lightning.loggers.logger.rank_zero_experiment` instead.
  return new_rank_zero_deprecation(*args, **kwargs)


In [2]:
import numpy as np
import pandas as pd

In [3]:
adata = sc.read_h5ad("/home/gaurav/Gaurav/Berlin/Deep_Learning/pytorchVAE/Larry_data/Neutrophil_monocyte/larry_hema_training_data.h5ad")

In [4]:
genes_to_remove = np.array(['Ngp', 'S100a8', 'S100a9'])
genes = np.array(adata.var_names)
train_genes = np.setdiff1d(genes,genes_to_remove)
adata = adata[:,train_genes]

In [5]:
adata_segment_22 = adata[adata.obs["segment"] == 22] #Monocyte
adata_segment_25 = adata[adata.obs["segment"] == 25] #Neutrophil 

In [6]:
training_data = adata_segment_22[(adata_segment_22.obs["pseudotime"] <= 0.1) | (adata_segment_22.obs["pseudotime"] > 0.3)]

In [7]:
training_data.obs["cells"] = "Training data"

/tmp/ipykernel_22208/3339499810.py:1: ImplicitModificationWarning: Trying to modify attribute `.obs` of view, initializing view as actual.
  training_data.obs["cells"] = "Training data"


In [8]:
zebra_obj = deep_predictor.DeepPredictor(training_data,latent_dim=100,hidden_layers=[800,800],workers=8,parameters=[0,0,0,1,1],likelihood="nb",batch_size=100)

/home/gaurav/anaconda3/envs/project/lib/python3.10/site-packages/torch/cuda/__init__.py:82: UserWarning: CUDA initialization: The NVIDIA driver on your system is too old (found version 10010). Please update your GPU driver by downloading and installing a new version from the URL: http://www.nvidia.com/Download/index.aspx Alternatively, go to: https://pytorch.org to install a PyTorch version that has been compiled with your version of the CUDA driver. (Triggered internally at  /opt/conda/conda-bld/pytorch_1646755853668/work/c10/cuda/CUDAFunctions.cpp:112.)
  return torch._C._cuda_getDeviceCount() > 0


In [9]:
zebra_obj.train(epochs=200)

GPU available: False, used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs

  | Name  | Type     | Params
-----------------------------------
0 | model | VAEmodel | 26.4 M
-----------------------------------
26.4 M    Trainable params
0         Non-trainable params
26.4 M    Total params
105.647   Total estimated model params size (MB)
/home/gaurav/anaconda3/envs/project/lib/python3.10/site-packages/pytorch_lightning/trainer/trainer.py:1892: PossibleUserWarning: The number of training batches (10) is smaller than the logging interval Trainer(log_every_n_steps=50). Set a lower value for log_every_n_steps if you want to see logs for the training epoch.
  rank_zero_warn(


Training: 0it [00:00, ?it/s]

`Trainer.fit` stopped: `max_epochs=200` reached.


In [10]:
zebra_obj.save_model(path="/home/gaurav/Gaurav/Berlin/Deep_Learning/Project/models/NB_MMD/withouttestdata/Hema_monocyte_trained_without_dominant_genes/model_200.pt")